# 1️⃣ Clone MeshMind Repository
This cell clones the MeshMind repository into Colab and navigates into it.  
All subsequent setup steps will operate from this directory.

In [ ]:
!git clone https://github.com/dimp01/MeshMind /content/MeshMind/
%cd /content/MeshMind/
!git pull

# 2️⃣ Install System Dependencies
PyVista and VTK require OpenGL libraries and a virtual framebuffer for headless rendering.
This cell installs the necessary Linux packages.


In [ ]:
!apt-get update
!apt-get install -y libgl1 libglib2.0-0 xvfb mesa-utils

# 3️⃣ Install Python Dependencies
Install all required Python packages from `requirements.txt`.  
This includes Streamlit, PyTorch, diffusers, PyVista, stpyvista, and more.


In [ ]:
!pip install --upgrade pip
!pip install -r requirements.txt

# 4️⃣ Run Setup
If `.env` file does not exist, this cell runs `setup.py` to configure:
- GEMINI_API_KEY
- NGROK_AUTH_TOKEN

This ensures the app has the required credentials.

In [ ]:
import os
if not os.path.exists(".env"):
    print("⚡ .env file not found. Running setup.py to create it.")
    !python setup.py
else:
    print("✅ .env file found. Skipping setup.py.")

# 5️⃣ Load Environment Variables
Load `.env` file so the GEMINI_API_KEY and NGROK_AUTH_TOKEN are available in the environment.

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # Automatically sets os.environ

# 6️⃣ Configure PyVista for Headless Rendering
PyVista requires off-screen rendering in Colab/Docker.  
This prevents OpenGL errors when running Streamlit.

In [ ]:
import pyvista as pv
pv.start_xvfb()  # off-screen rendering

# 7️⃣ Start Streamlit and ngrok
This cell:
1. Launches Streamlit in the background.
2. Opens a public ngrok tunnel.
3. Prints the public URL for accessing the app.

In [ ]:
from pyngrok import ngrok
import subprocess
import time
import os

# Kill any previous tunnels
ngrok.kill()

port = 8501
os.environ["STREAMLIT_SERVER_HEADLESS"] = "true"

# Launch Streamlit
cmd = f"streamlit run main.py --server.port {port} --server.headless true"
process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Open ngrok tunnel
public_url = ngrok.connect(port)
print("🌐 Streamlit is running at:", public_url)

# Give Streamlit a few seconds to start
time.sleep(5)

# 8️⃣ Stop Streamlit and ngrok
Use this cell to terminate all running Streamlit servers and ngrok tunnels.
Useful for restarting the app or cleaning up resources.

In [ ]:
from pyngrok import ngrok
import os

ngrok.kill()             # Close ngrok tunnels
!pkill -f streamlit      # Kill all Streamlit processes
print("✅ Streamlit + ngrok terminated.")